In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22177309
paper_name = 'suzuki_harashima_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [19]:
original_data = pd.read_csv('raw_data/Table1.txt', sep='\t')

In [20]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 107 x 3


In [21]:
original_data.head()

,ORF,Description,Lactic acid sensitivity
0,ADA2,General transcriptional adaptor or co-activator,- -
1,ALF1,Protein implicated in folding of alpha tubulin,- - - -
2,ANP1,Protein required for protein glycosylation in ...,-
3,ARG82,Arginine metabolism transcription factor,-
4,ARO1,Arom pentafunctional enzyme,- - -


In [22]:
original_data['gene'] = original_data['ORF'].astype(str)

In [23]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [24]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Description, Lactic acid sensitivity, gene, orf]
Index: []


In [26]:
original_data['data'] = original_data['Lactic acid sensitivity'].apply(lambda x: -len(x.replace(' ','')))

In [27]:
original_data.set_index('orf', inplace=True)

In [28]:
original_data = original_data[['data']].copy()

In [29]:
original_data = original_data.groupby(original_data.index).mean()

In [30]:
original_data.shape

(107, 1)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [33]:
dataset_ids = [21839]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,21839
data_type,value
orf,
YBL058W,-2
YBR011C,-2
YBR035C,-4
YBR036C,-2
YBR068C,-2


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21839
,data_type,value
gene_id,orf,
146,YBL058W,-2
207,YBR011C,-2
231,YBR035C,-4
232,YBR036C,-2
264,YBR068C,-2


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      21839           
data_type       value     valuez
gene_id orf                     
146     YBL058W    -2  -5.045048
207     YBR011C    -2  -5.045048
231     YBR035C    -4 -10.090097
232     YBR036C    -2  -5.045048
264     YBR068C    -2  -5.045048

# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 22177309...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.10it/s]

Updating the data_modified_on field...
